In [ ]:
import json
import re
from transformers import AutoTokenizer

# Cargar el JSON etiquetado (el archivo que subiste)
with open('training_data.json', 'r', encoding='utf-8') as f:
    data = json.load(f)['TRAINING_DATA']

# 1. Definir el mapeo de etiquetas
# Todas las etiquetas de tu JSON. 'O' significa 'Outside' (Fuera de entidad).
LABEL_NAMES = ["O", "INGREDIENTE", "CANTIDAD", "UNIDAD", "UTENSILIO", "ACCION", "TIEMPO","PREPARACION",
    "INSTRUCCION", "COLOR","ESTADO", "FORMA", "MEDIDA", "SUPERFICIE", "TECNICA", "TEMP", "TEXTURA"
]

# Crear mapeo B- y I- (Beginning e Inside)
tag_list = ['O']
for label in LABEL_NAMES[1:]:
    tag_list.append(f'B-{label}')
    tag_list.append(f'I-{label}')
tag_to_id = {tag: i for i, tag in enumerate(tag_list)}
id_to_tag = {i: tag for tag, i in tag_to_id.items()}

def convert_json_to_iob(example, tokenizer):
    """Convierte un ejemplo de tu formato JSON a tokens y etiquetas IOB."""
    text = example['text']
    entities = example['entities']

    # Tokenización
    tokens = tokenizer(text, return_offsets_mapping=True, truncation=False, add_special_tokens=True)
    offsets = tokens['offset_mapping']

    # Inicializar todas las etiquetas como 'O'
    labels = [tag_to_id['O']] * len(offsets)

    # Crear un mapa de caracteres a etiquetas
    # Inicializar todos los caracteres como 'O'
    char_labels = ['O'] * len(text)

    # Marcar los rangos de caracteres con sus etiquetas
    for entity in entities:
        start_char, end_char = entity['start'], entity['end']
        label = entity['label'].upper()

        if f'B-{label}' not in tag_to_id:
            print(f"Advertencia: Etiqueta '{label}' no está en LABEL_NAMES. Saltando...")
            continue

        # Marcar el primer carácter como B-, el resto como I-
        if start_char < len(char_labels):
            char_labels[start_char] = f'B-{label}'
        for i in range(start_char + 1, min(end_char, len(char_labels))):
            char_labels[i] = f'I-{label}'

    # Asignar etiquetas a tokens basándose en su offset
    for i, (offset_start, offset_end) in enumerate(offsets):
        if offset_start is None or offset_end is None:
            continue

        # Verificar qué etiqueta tiene la mayoría de los caracteres en este token
        if offset_start < offset_end:  # Token no especial
            token_chars = char_labels[offset_start:offset_end]

            # Contar las etiquetas en los caracteres de este token
            from collections import Counter
            counter = Counter(token_chars)

            if counter:
                # Eliminar 'O' del conteo para priorizar entidades
                if 'O' in counter:
                    del counter['O']

                if counter:
                    # Tomar la etiqueta más común
                    most_common_label = counter.most_common(1)[0][0]

                    # Si es una etiqueta B- o I-
                    if most_common_label != 'O':
                        # Determinar si este token debería ser B- o I-
                        # Si el primer carácter del token es B-, entonces este token es B-
                        if char_labels[offset_start].startswith('B-'):
                            labels[i] = tag_to_id[char_labels[offset_start]]
                        else:
                            # Buscar si hay alguna etiqueta B- en este token
                            has_b_label = any(label.startswith('B-') for label in token_chars if label != 'O')
                            if has_b_label:
                                # Encontrar la primera etiqueta B-
                                for label in token_chars:
                                    if label.startswith('B-'):
                                        labels[i] = tag_to_id[label]
                                        break
                            else:
                                # Usar la etiqueta más común
                                labels[i] = tag_to_id[most_common_label]

    return {
        'tokens': tokens.tokens(),
        'input_ids': tokens['input_ids'],
        'attention_mask': tokens['attention_mask'],
        'labels': labels
    }

# Seleccionar un modelo Transformer multilingüe (ideal para el español)
MODEL_CHECKPOINT = "dccuchile/bert-base-spanish-wwm-uncased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)

# Aplicar la conversión a todos los datos de entrenamiento
training_examples = [convert_json_to_iob(example, tokenizer) for example in data]
# Filtrar ejemplos que podrían ser problemáticos o que solo contienen tokens especiales
training_examples = [ex for ex in training_examples if any(l != tag_to_id['O'] for l in ex['labels'])]

print(f"Número total de etiquetas únicas IOB: {len(tag_list)}")
print(f"Ejemplos de entrenamiento generados: {len(training_examples)}")

# Ejemplo de un dato procesado
print("\n--- Ejemplo de Dato Procesado ---")
print("Tokens:", training_examples[0]['tokens'][:10])
print("Etiquetas (IDs):", training_examples[0]['labels'][:10])
print("Etiquetas (IOB):", [id_to_tag[l] for l in training_examples[0]['labels'][:10]])

# Verificar la corrección
test_example = data[0]
converted = convert_json_to_iob(test_example, tokenizer)

print("\n=== VERIFICACIÓN DE CORRECCIÓN ===")
print("Texto:", test_example['text'][:50], "...")
print("\nTokens y etiquetas:")
for i, (token, label_id) in enumerate(zip(converted['tokens'][:20], converted['labels'][:20])):
    print(f"{i:2d}. {token:15s} -> {id_to_tag[label_id]}")